# Day 5: ReAct Agents with Tools and Function Calling

**Learning Goals:**
- Understand the ReAct (Reasoning + Acting) pattern — Thought → Action → Observation
- Create custom tools using the `@tool` decorator
- Build ReAct agents with `create_react_agent` and `AgentExecutor`
- Use function calling / `bind_tools()` for structured tool inputs
- Add conversation memory to agents for multi-turn sessions
- Chain multi-step tool calls for complex queries
- Observe agent traces in LangSmith

**Time:** 2–3 hours

---

## What We'll Build Today

1. **Custom Tools** — Calculator, word counter, mock weather, text summarizer
2. **ReAct Agent** — `create_react_agent` + `AgentExecutor` with verbose traces
3. **Tool Calling Agent** — `bind_tools()` + Pydantic schemas for structured I/O
4. **Multi-Tool Research Assistant** — DuckDuckGo search + custom tools
5. **Agent with Memory** — Persistent context across conversation turns
6. **Multi-Step Reasoning** — Chained tool calls for complex queries
7. **LangSmith Observability** — Traces, latency, token usage

---

## 🧠 The ReAct Pattern

```
User Question
     ↓
  [Thought]  → LLM reasons about what to do next
     ↓
  [Action]   → LLM selects a tool and input
     ↓
 [Observation] → Tool executes and returns result
     ↓
  [Thought]  → LLM reasons about the observation
     ↓
 ... (loop until answer is ready) ...
     ↓
 [Final Answer]
```

This loop lets the agent **plan**, **act**, and **adapt** — just like a human researcher.


## Part 1: Environment Setup

Load API keys, configure LangSmith tracing, and initialize the LLM.
> **Note:** We use `temperature=0` for agents — deterministic reasoning produces more reliable tool selections.

In [5]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
LANGCHAIN_API_KEY  = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING  = os.getenv("LANGCHAIN_TRACING_V2", "false") == "true"

# LangSmith env vars (used automatically by LangChain when set)
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"]      = LANGCHAIN_API_KEY
    os.environ["LANGCHAIN_TRACING_V2"]   = "true"
    os.environ["LANGCHAIN_PROJECT"]      = "personal-ai-day5"

print("✅ OpenRouter key loaded" if OPENROUTER_API_KEY else "⚠️  Missing OPENROUTER_API_KEY")
print("✅ LangSmith tracing enabled" if LANGCHAIN_TRACING else "ℹ️  LangSmith tracing disabled")


✅ OpenRouter key loaded
✅ LangSmith tracing enabled


In [6]:
from langchain_openai import ChatOpenAI

# temperature=0 → deterministic reasoning (critical for reliable agent behaviour)
llm = ChatOpenAI(
    model="openai/gpt-3.5-turbo",
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0,
)

# Quick smoke-test
response = llm.invoke("Say 'Agent ready!' in exactly 3 words.")
print("LLM response:", response.content)
print("✅ LLM initialized and responding")


LLM response: Agent ready now!
✅ LLM initialized and responding


## Part 2: Import Agent Libraries

Import everything we need for building ReAct agents, defining tools, and adding memory.

In [7]:
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# In LangChain 1.x, agent helpers live in langchain_classic
from langchain_classic.agents import create_tool_calling_agent, create_react_agent, AgentExecutor

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# For built-in search tool
from langchain_community.tools import DuckDuckGoSearchResults

# For structured tool schemas
from pydantic import BaseModel, Field
from typing import Optional
import math
import json

print("✅ All agent libraries imported successfully")


✅ All agent libraries imported successfully


## Part 3: Defining Custom Tools

Tools are the **hands** of an agent. Each tool:
- Has a **name** (used in the Thought/Action trace)
- Has a **docstring** (the LLM reads this to decide *when* to use it)
- Accepts typed inputs and returns a string result

> 💡 **Key insight**: The docstring is your tool's "API contract" with the LLM. Be descriptive!

### 3a. Basic Custom Tools

In [8]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression and return the numeric result.
    Use this for any arithmetic, algebra, or math calculations.
    Examples: '2 + 2', '42 * 7', 'sqrt(144)', '(10 + 5) * 3 / 2'
    """
    try:
        # Allow safe math functions
        safe_globals = {"__builtins__": {}, "sqrt": math.sqrt, "pi": math.pi,
                        "e": math.e, "sin": math.sin, "cos": math.cos, "log": math.log}
        result = eval(expression, safe_globals)
        return f"{expression} = {result}"
    except Exception as ex:
        return f"Error evaluating '{expression}': {ex}"


@tool
def word_counter(text: str) -> str:
    """
    Count the number of words, characters, and sentences in a given text.
    Use this when the user asks about text statistics or word counts.
    """
    words      = len(text.split())
    characters = len(text)
    sentences  = len([s for s in text.split('.') if s.strip()])
    return (
        f"Text analysis:\n"
        f"  Words:      {words}\n"
        f"  Characters: {characters}\n"
        f"  Sentences:  {sentences}"
    )


@tool
def get_weather(city: str) -> str:
    """
    Get the current weather for a given city.
    Returns temperature, conditions, and humidity.
    Use this when the user asks about weather in a specific location.
    """
    # Defensive: keep only the first line and strip quotes
    city_clean = city.splitlines()[0].strip().strip("'\"")

    # Mock data — replace with a real weather API in production
    mock_weather = {
        "london":    {"temp": "12°C", "condition": "Cloudy",  "humidity": "78%"},
        "new york":  {"temp": "18°C", "condition": "Sunny",   "humidity": "55%"},
        "tokyo":     {"temp": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "sydney":    {"temp": "25°C", "condition": "Clear",   "humidity": "50%"},
        "paris":     {"temp": "15°C", "condition": "Rainy",   "humidity": "82%"},
    }
    key = city_clean.lower().strip()
    if key in mock_weather:
        w = mock_weather[key]
        return f"Weather in {city_clean.title()}: {w['temp']}, {w['condition']}, Humidity: {w['humidity']}"
    return f"Weather data not available for '{city_clean}'. Try: London, New York, Tokyo, Sydney, Paris."


@tool
def text_summarizer(text: str) -> str:
    """
    Produce a brief 1-2 sentence summary of the provided text.
    Use this to condense long passages into key points.
    """
    words = text.split()
    if len(words) <= 20:
        return f"Text is already short: {text}"
    # Simple extractive summary: first sentence + word count note
    first_sentence = text.split('.')[0].strip()
    return f"Summary: {first_sentence}. (Original: {len(words)} words)"


# Collect all custom tools
custom_tools = [calculator, word_counter, get_weather, text_summarizer]

print("✅ Custom tools defined:")
for t in custom_tools:
    print(f"  🔧 {t.name}: {t.description[:60]}...")


✅ Custom tools defined:
  🔧 calculator: Evaluate a mathematical expression and return the numeric re...
  🔧 word_counter: Count the number of words, characters, and sentences in a gi...
  🔧 get_weather: Get the current weather for a given city.
Returns temperatur...
  🔧 text_summarizer: Produce a brief 1-2 sentence summary of the provided text.
U...


### 3b. Test Tools Individually

Always test tools in isolation before attaching them to an agent. This makes debugging much easier.

In [9]:
# ---- Calculator ----
print("=== calculator ===")
print(calculator.invoke("42 * 7"))
print(calculator.invoke("sqrt(144)"))
print(calculator.invoke("(10 + 5) * 3 / 2"))

# ---- Word Counter ----
print("\n=== word_counter ===")
sample = "The quick brown fox jumps over the lazy dog. It was a sunny afternoon."
print(word_counter.invoke(sample))

# ---- Weather ----
print("\n=== get_weather ===")
print(get_weather.invoke("London"))
print(get_weather.invoke("Tokyo"))
print(get_weather.invoke("Berlin"))  # not in mock data

# ---- Summarizer ----
print("\n=== text_summarizer ===")
long_text = ("LangChain is a framework for building applications powered by large language models. "
             "It provides abstractions for chains, agents, memory, and retrieval systems. "
             "Developers use it to build chatbots, RAG pipelines, and autonomous agents.")
print(text_summarizer.invoke(long_text))


=== calculator ===
42 * 7 = 294
sqrt(144) = 12.0
(10 + 5) * 3 / 2 = 22.5

=== word_counter ===
Text analysis:
  Words:      14
  Characters: 70
  Sentences:  2

=== get_weather ===
Weather in London: 12°C, Cloudy, Humidity: 78%
Weather in Tokyo: 22°C, Partly cloudy, Humidity: 65%
Weather data not available for 'Berlin'. Try: London, New York, Tokyo, Sydney, Paris.

=== text_summarizer ===
Summary: LangChain is a framework for building applications powered by large language models. (Original: 33 words)


## Part 4: Building a ReAct Agent

### 4a. The Classic ReAct Agent (`create_react_agent`)

The classic ReAct agent uses a **text-based** prompt with explicit `Thought:`, `Action:`, `Action Input:`, and `Observation:` markers. The LLM generates its reasoning as text, which the framework parses.

We pull the standard ReAct prompt from LangChain Hub.

In [10]:
from langsmith import Client
from langchain_core.prompts import PromptTemplate

client = Client()

# Pull the standard ReAct prompt (hwchase17/react)
# This contains the classic Thought/Action/Observation template
base_prompt = client.pull_prompt("hwchase17/react")

# Reinforce the exact format expected by the ReAct parser
react_prompt = PromptTemplate(
    input_variables=base_prompt.input_variables,
    template=base_prompt.template
    + "\n\nIMPORTANT RULES:\n"
    + "- Use the exact format below (including labels).\n"
    + "- Only ONE Action per step. Wait for the Observation before the next Thought.\n"
    + "- Put the tool input on its own line after 'Action Input:'.\n"
    + "- Do NOT include the 'Observation:' label or any tool output. The system will add it.\n\n"
    + "Thought: <your reasoning>\n"
    + "Action: <tool name>\n"
    + "Action Input: <tool input>\n",
    partial_variables=base_prompt.partial_variables,
)

print("✅ ReAct prompt pulled from LangChain Hub")
print("\nPrompt template preview:")
print(react_prompt.template[:400], "...")


✅ ReAct prompt pulled from LangChain Hub

Prompt template preview:
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observ ...


In [11]:
# Build the ReAct agent
react_agent = create_react_agent(
    llm=llm,
    tools=custom_tools,
    prompt=react_prompt,
)

# Wrap in AgentExecutor — this manages the Thought/Action/Observation loop
react_executor = AgentExecutor(
    agent=react_agent,
    tools=custom_tools,
    verbose=True,          # Print the full reasoning trace
    max_iterations=5,      # Safety cap — stop after 5 loops
    handle_parsing_errors=True,   # Gracefully handle malformed LLM output
)

print("✅ ReAct agent and executor ready")


✅ ReAct agent and executor ready


In [12]:
# --- Run 1: Simple single-tool query ---
print("=" * 60)
print("Query: What is 42 * 7?")
print("=" * 60)

result = react_executor.invoke({"input": "What is 42 * 7?"})
print("\n🎯 Final Answer:", result["output"])


Query: What is 42 * 7?


> Entering new AgentExecutor chain...
Thought: I need to calculate the product of 42 and 7.
Action: calculator
Action Input: '42 * 7''42 * 7' = 42 * 7Observation: 42 * 7 = 294
Thought: I now know the final answer.

Final Answer: The result of 42 * 7 is 294.

> Finished chain.

🎯 Final Answer: The result of 42 * 7 is 294.


In [13]:
# --- Run 2: Multi-tool query (weather + calculator) ---
print("=" * 60)
print("Query: What's the weather in London, and what is 15% of 240?")
print("=" * 60)

result = react_executor.invoke({
    "input": "What's the weather in London, and what is 15% of 240?"
})
print("\n🎯 Final Answer:", result["output"])


Query: What's the weather in London, and what is 15% of 240?


> Entering new AgentExecutor chain...
Thought: I should first get the weather in London using the get_weather tool.
Action: get_weather
Action Input: London
Thought: Now, I need to calculate 15% of 240 using the calculator tool.
Action: calculator
Action Input: 0.15 * 240Weather in London: 12°C, Cloudy, Humidity: 78%Final Answer: Weather in London: 12°C, Cloudy, Humidity: 78%. 15% of 240 is 36.

> Finished chain.

🎯 Final Answer: Weather in London: 12°C, Cloudy, Humidity: 78%. 15% of 240 is 36.


## Part 5: Function Calling Agent with Structured Inputs

### 5a. Why Use Tool Calling?

The **tool calling** (function calling) approach is the **modern, recommended** pattern:

| Feature | ReAct (text-based) | Tool Calling |
|---|---|---|
| Tool selection | LLM writes `Action: tool_name` | LLM emits structured JSON |
| Input parsing | Text parsing (fragile) | Native JSON schema (robust) |
| Multi-tool in one step | ❌ One at a time | ✅ Parallel calls |
| Model requirement | Any LLM | Function-calling capable LLM |

With OpenRouter's `openai/gpt-3.5-turbo`, function calling is fully supported.

### 5b. Pydantic Schemas for Structured Inputs

In [14]:
from pydantic import BaseModel, Field

# --- Pydantic schema for a structured calculator tool ---
class CalculatorInput(BaseModel):
    expression: str = Field(description="The mathematical expression to evaluate, e.g. '2 + 2' or 'sqrt(16)'")

# --- Pydantic schema for weather lookup ---
class WeatherInput(BaseModel):
    city: str = Field(description="The name of the city to get weather for, e.g. 'London' or 'Tokyo'")

# Redefine tools with explicit args_schema for structured function calling
@tool(args_schema=CalculatorInput)
def structured_calculator(expression: str) -> str:
    """Evaluate a mathematical expression. Use for any arithmetic or math operations."""
    try:
        safe_globals = {"__builtins__": {}, "sqrt": math.sqrt, "pi": math.pi,
                        "e": math.e, "sin": math.sin, "cos": math.cos}
        result = eval(expression, safe_globals)
        return f"{expression} = {result}"
    except Exception as ex:
        return f"Error: {ex}"

@tool(args_schema=WeatherInput)
def structured_weather(city: str) -> str:
    """Get current weather conditions for a city. Returns temperature and conditions."""
    mock = {
        "london":   {"temp": "12°C", "condition": "Cloudy",  "humidity": "78%"},
        "new york": {"temp": "18°C", "condition": "Sunny",   "humidity": "55%"},
        "tokyo":    {"temp": "22°C", "condition": "Partly cloudy", "humidity": "65%"},
        "sydney":   {"temp": "25°C", "condition": "Clear",   "humidity": "50%"},
        "paris":    {"temp": "15°C", "condition": "Rainy",   "humidity": "82%"},
    }
    key = city.lower().strip()
    if key in mock:
        w = mock[key]
        return f"{city.title()}: {w['temp']}, {w['condition']}, Humidity {w['humidity']}"
    return f"No data for '{city}'."

# Show the auto-generated JSON schema the LLM will see
print("📐 CalculatorInput JSON schema:")
print(json.dumps(CalculatorInput.model_json_schema(), indent=2))


📐 CalculatorInput JSON schema:
{
  "properties": {
    "expression": {
      "description": "The mathematical expression to evaluate, e.g. '2 + 2' or 'sqrt(16)'",
      "title": "Expression",
      "type": "string"
    }
  },
  "required": [
    "expression"
  ],
  "title": "CalculatorInput",
  "type": "object"
}


In [15]:
# Bind tools to the LLM — the LLM now "knows" these tools and their schemas
llm_with_tools = llm.bind_tools([structured_calculator, structured_weather, word_counter, text_summarizer])

# Direct function-calling test (no agent loop yet — just raw LLM)
test_message = HumanMessage(content="What is 99 * 99? Also what is the weather in Tokyo?")
response = llm_with_tools.invoke([test_message])

print("LLM response type:", type(response).__name__)
print("\nTool calls requested by LLM:")
for tc in response.tool_calls:
    print(f"  🔧 Tool: {tc['name']}")
    print(f"     Args: {tc['args']}")


LLM response type: AIMessage

Tool calls requested by LLM:
  🔧 Tool: structured_calculator
     Args: {'expression': '99 * 99'}
  🔧 Tool: structured_weather
     Args: {'city': 'Tokyo'}


### 5c. Tool Calling Agent with `create_tool_calling_agent`

This is the **preferred modern approach** — it uses the LLM's native function-calling capability instead of text parsing.

The prompt must include a `MessagesPlaceholder("agent_scratchpad")` where the agent records its tool call history.

In [16]:
all_custom_tools = [structured_calculator, structured_weather, word_counter, text_summarizer]

# Build the prompt — must include MessagesPlaceholder("agent_scratchpad")
tool_calling_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful AI assistant with access to tools. "
     "Always use the appropriate tool when you need to look up information or perform calculations. "
     "Be concise and accurate in your final answers."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),  # REQUIRED: stores tool call history
])

# Create the tool-calling agent
tool_calling_agent = create_tool_calling_agent(
    llm=llm,
    tools=all_custom_tools,
    prompt=tool_calling_prompt,
)

# Wrap in AgentExecutor
tool_calling_executor = AgentExecutor(
    agent=tool_calling_agent,
    tools=all_custom_tools,
    verbose=True,
    max_iterations=6,
    handle_parsing_errors=True,
    return_intermediate_steps=True,  # Capture full reasoning chain
)

print("✅ Tool-calling agent ready")


✅ Tool-calling agent ready


In [17]:
# Run the tool-calling agent
result = tool_calling_executor.invoke({
    "input": "What is the weather in Paris, and what is 256 / 16?"
})

print("\n" + "=" * 60)
print("🎯 Final Answer:", result["output"])




> Entering new AgentExecutor chain...

Invoking: `structured_weather` with `{'city': 'Paris'}`


Paris: 15°C, Rainy, Humidity 82%
Invoking: `structured_calculator` with `{'expression': '256 / 16'}`


256 / 16 = 16.0The weather in Paris is 15°C with rainy conditions and 82% humidity. 

The result of 256 divided by 16 is 16.0.

> Finished chain.

🎯 Final Answer: The weather in Paris is 15°C with rainy conditions and 82% humidity. 

The result of 256 divided by 16 is 16.0.


## Part 6: Adding a Built-in Search Tool (DuckDuckGo)

Real agents need access to **live information**. `DuckDuckGoSearchRun` is a zero-config search tool from `langchain-community` that queries the web without requiring an API key.

### 6a. Multi-Tool Research Assistant

Let's build an agent that can:
- 🔍 Search the web for real-time information
- 🔢 Perform calculations on the results
- 🌤 Check weather conditions
- 📝 Summarize findings

In [2]:
# Install compatible version of duckduckgo-search
# Note: Using ddgs package which is the maintained version
%pip install -q ddgs

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.tools import DuckDuckGoSearchResults

# Initialize the DuckDuckGo search tool with api backend
search_tool = DuckDuckGoSearchResults()

# Quick test
print("\nTesting DuckDuckGo search...")
try:
    search_result = search_tool.invoke("LangChain latest version 2024")
    print("Search result (first 300 chars):")
    print(str(search_result)[:300], "...\n")
    print(f"✅ Search tool ready: {search_tool.name}")
    print(f"   Description: {search_tool.description}")
except Exception as e:
    print(f"Search test error: {e}")
    print("Note: Search may work better in actual agent use.")


Testing DuckDuckGo search...
Search result (first 300 chars):
snippet: Official updates from the LangChain team including product updates., title: Latest Announcements topics - LangChain Forum, link: https://forum.langchain.com/c/announcements/15, snippet: 🦜🔗 LangChain interfaces to Google's suite of AI products (Gemini & Vertex AI) - langchain -ai/ langchain  ...

✅ Search tool ready: duckduckgo_results_json
   Description: A wrapper around Duck Duck Go Search. Useful for when you need to answer questions about current events. Input should be a search query.


In [19]:
# --- Research Assistant: all tools combined ---
research_tools = [search_tool, structured_calculator, structured_weather, word_counter, text_summarizer]

research_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert research assistant. You have access to web search, "
     "weather lookups, a calculator, a word counter, and a text summarizer. "
     "For factual questions, always use the search tool first. "
     "Show your reasoning and cite your sources where possible. "
     "Be thorough but concise."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

research_agent = create_tool_calling_agent(llm=llm, tools=research_tools, prompt=research_prompt)

research_executor = AgentExecutor(
    agent=research_agent,
    tools=research_tools,
    verbose=True,
    max_iterations=8,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

print("✅ Research assistant agent ready with tools:")
for t in research_tools:
    print(f"   🔧 {t.name}")


✅ Research assistant agent ready with tools:
   🔧 duckduckgo_results_json
   🔧 structured_calculator
   🔧 structured_weather
   🔧 word_counter
   🔧 text_summarizer


In [20]:
# Research query: requires search + calculator
research_result = research_executor.invoke({
    "input": "What is the population of Japan, and if it decreased by 0.5% annually, "
             "what would the population be after 10 years? Show your calculation."
})

print("\n" + "=" * 60)
print("🎯 Final Answer:")
print(research_result["output"])




> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'current population of Japan'}`


snippet: 15 Jul 2025 · As of April 2025, Japan's population was roughly 123.4 million people, and peaked at 128.5 million people in 2010. It is the 6th-most populous country in Asia, ... Historical overview · Population, title: Demographics of Japan - Wikipedia, link: https://en.wikipedia.org/wiki/Demographics_of_Japan, snippet: 5 days ago · With a population of almost 123 million as of 2026, it is the world's 11th most populous country. Tokyo is the country's capital and largest city. Japan. 日本国., title: Japan - Wikipedia, link: https://en.wikipedia.org/wiki/Japan, snippet: 15 Dec 2025 · The total population in Japan was estimated at 123.8 million people in 2024, according to the latest census figures and projections from Trading Economics. ..., title: Japan Population - Trading Economics, link: https://tradingeconomics.com/japan/population, snippet: 1 Jan 202

## Part 7: Agent with Conversation Memory

So far our agents have **no memory** — each call starts fresh. Let's add `ConversationBufferMemory` so the agent can remember what was said earlier in the session.

### How it works:
- `ConversationBufferMemory` stores all messages in a buffer
- `RunnableWithMessageHistory` attaches a session-scoped history store
- The prompt's `MessagesPlaceholder("chat_history")` injects previous messages

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Prompt with chat history placeholder
memory_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant with tools. "
     "Remember previous messages in the conversation to provide contextual answers."),
    MessagesPlaceholder("chat_history"),   # <-- previous turns injected here
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

memory_agent = create_tool_calling_agent(
    llm=llm,
    tools=all_custom_tools,
    prompt=memory_prompt,
)

memory_executor = AgentExecutor(
    agent=memory_agent,
    tools=all_custom_tools,
    verbose=False,    # Keep output clean for multi-turn demo
    max_iterations=5,
    handle_parsing_errors=True,
)

# In-memory session store: session_id → ChatMessageHistory
session_store: dict[str, ChatMessageHistory] = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    """Return (or create) the message history for a given session."""
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

# Wrap executor with message history management
agent_with_memory = RunnableWithMessageHistory(
    memory_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ Agent with conversation memory ready")


✅ Agent with conversation memory ready


In [22]:
# ---- Multi-turn conversation demo ----
config = {"configurable": {"session_id": "demo-session-001"}}

def chat(message: str) -> str:
    """Send a message to the memory-enabled agent and return its response."""
    result = agent_with_memory.invoke({"input": message}, config=config)
    return result["output"]

print("=== Multi-turn Conversation ===\n")

# Turn 1
turn1 = "My name is Rohan. What is 15 * 8?"
print(f"👤 User: {turn1}")
resp1 = chat(turn1)
print(f"🤖 Agent: {resp1}\n")

# Turn 2 — tests if agent remembers the name
turn2 = "What was the result from my last calculation? Also, do you remember my name?"
print(f"👤 User: {turn2}")
resp2 = chat(turn2)
print(f"🤖 Agent: {resp2}\n")

# Turn 3 — new calculation referencing previous context
turn3 = "Now multiply that result by 3, and tell me the weather in New York."
print(f"👤 User: {turn3}")
resp3 = chat(turn3)
print(f"🤖 Agent: {resp3}\n")

# Show memory buffer
print("=== Session Memory Buffer ===")
history = get_session_history("demo-session-001")
print(f"Stored {len(history.messages)} messages in session 'demo-session-001'")
for i, msg in enumerate(history.messages):
    role = "Human" if msg.__class__.__name__ == "HumanMessage" else "AI"
    print(f"  [{i+1}] {role}: {str(msg.content)[:80]}...")


=== Multi-turn Conversation ===

👤 User: My name is Rohan. What is 15 * 8?
🤖 Agent: 15 multiplied by 8 is equal to 120.

👤 User: What was the result from my last calculation? Also, do you remember my name?
🤖 Agent: The result from your last calculation was 120 (15 * 8). And yes, I remember your name is Rohan.

👤 User: Now multiply that result by 3, and tell me the weather in New York.
🤖 Agent: The result of multiplying 120 by 3 is 360. The weather in New York is currently 18°C, sunny, with a humidity of 55%.

=== Session Memory Buffer ===
Stored 6 messages in session 'demo-session-001'
  [1] Human: My name is Rohan. What is 15 * 8?...
  [2] AI: 15 multiplied by 8 is equal to 120....
  [3] Human: What was the result from my last calculation? Also, do you remember my name?...
  [4] AI: The result from your last calculation was 120 (15 * 8). And yes, I remember your...
  [5] Human: Now multiply that result by 3, and tell me the weather in New York....
  [6] AI: The result of multiplying 1

## Part 8: Multi-Step Reasoning (Chaining Tool Calls)

This is the core power of agents — **chaining multiple tools together** to answer questions that no single tool could handle alone.

The agent will:
1. Search for a fact 🔍
2. Parse a number from the result 🔢
3. Perform a calculation on it 📐
4. Summarize the final answer 📝

In [23]:
# Multi-step query — requires at least 3 tool calls
multistep_query = (
    "Search for how many countries are in the European Union, "
    "then calculate what 15% of that number is, "
    "and finally summarize your findings in one sentence."
)

print("=" * 60)
print(f"Multi-step Query:\n{multistep_query}")
print("=" * 60)

multistep_result = research_executor.invoke({"input": multistep_query})

# --- Inspect intermediate steps ---
print("\n📊 Intermediate Steps Breakdown:")
steps = multistep_result.get("intermediate_steps", [])
for i, (action, observation) in enumerate(steps, 1):
    print(f"\n  Step {i}:")
    print(f"    🔧 Tool Used: {action.tool}")
    print(f"    📥 Input:     {str(action.tool_input)[:80]}")
    print(f"    📤 Result:    {str(observation)[:100]}...")

print(f"\n  Total tool calls: {len(steps)}")
print("\n🎯 Final Answer:")
print(multistep_result["output"])


Multi-step Query:
Search for how many countries are in the European Union, then calculate what 15% of that number is, and finally summarize your findings in one sentence.


> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'number of countries in the European Union'}`


snippet: 1 month ago - EU15 includes the fifteen countries in the European Union from 1 January 1995 to 30 April 2004. The EU15 comprised Austria, Belgium, Denmark, Finland, France, Germany, Greece, Ireland, Italy, Luxembourg, Netherlands, Portugal, Spain, Sweden, and United Kingdom., title: Member state of the European Union - Wikipedia, link: https://en.wikipedia.org/wiki/Member_state_of_the_European_Union, snippet: 1 week ago - Maps of Nomenclature of Territorial Units for Statistics (NUTS) subdivisions (prior to 2018, including non-EU member states) ... There are nine countries that are recognised as candidates for membership: Albania, Bosnia and Herzegovina, Georgia, Moldova

## Part 9: Error Handling & Agent Safety

### 9a. Common Agent Failure Modes

1. **Infinite loops** — agent keeps calling tools without converging → `max_iterations`
2. **Parsing errors** — LLM output doesn't match expected format → `handle_parsing_errors`
3. **Tool errors** — tool raises an exception → wrap in try/except inside the tool
4. **Token limits** — very long tool outputs fill the context → summarize observations

### 9b. Early Stopping Strategies

In [24]:
# --- Demonstrate max_iterations safety cap ---
# Create a deliberately tight executor (max 2 iterations)
tight_executor = AgentExecutor(
    agent=tool_calling_agent,
    tools=all_custom_tools,
    verbose=True,
    max_iterations=2,                      # Stops after 2 tool calls
    early_stopping_method="generate",      # LLM generates a partial answer on stop
    handle_parsing_errors=True,
)

print("Testing with max_iterations=2 on a complex query...")
result = tight_executor.invoke({
    "input": "What is 10 + 20? Then what is 30 + 40? Then what is 70 + 80?"
})
print("\n🛑 Result (may be partial due to iteration cap):")
print(result["output"])


Testing with max_iterations=2 on a complex query...


> Entering new AgentExecutor chain...

Invoking: `structured_calculator` with `{'expression': '10 + 20'}`


10 + 20 = 30
Invoking: `structured_calculator` with `{'expression': '30 + 40'}`


30 + 40 = 70
Invoking: `structured_calculator` with `{'expression': '70 + 80'}`


70 + 80 = 150The results are:
1. 10 + 20 = 30
2. 30 + 40 = 70
3. 70 + 80 = 150

> Finished chain.

🛑 Result (may be partial due to iteration cap):
The results are:
1. 10 + 20 = 30
2. 30 + 40 = 70
3. 70 + 80 = 150


In [25]:
# --- Demonstrate graceful tool error handling ---
@tool
def risky_tool(value: str) -> str:
    """A tool that may fail — demonstrates handle_parsing_errors in action."""
    if value == "fail":
        raise ValueError("Intentional failure to demonstrate error handling!")
    return f"Success! Processed: {value}"

safe_executor = AgentExecutor(
    agent=create_tool_calling_agent(
        llm=llm,
        tools=[risky_tool, structured_calculator],
        prompt=tool_calling_prompt,
    ),
    tools=[risky_tool, structured_calculator],
    verbose=True,
    handle_parsing_errors=True,   # Catches malformed LLM output
    max_iterations=3,
)

print("Testing error recovery...")
try:
    result = safe_executor.invoke({"input": "Use risky_tool with value 'fail', then calculate 5 + 5."})
    print("\n🎯 Agent recovered and answered:", result["output"])
except Exception as e:
    print(f"\n⚠️  Unrecoverable error: {e}")


Testing error recovery...


> Entering new AgentExecutor chain...

Invoking: `risky_tool` with `{'value': 'fail'}`



⚠️  Unrecoverable error: Intentional failure to demonstrate error handling!


## Part 10: Observing Traces in LangSmith

LangSmith records every step of every agent run — thoughts, tool calls, token usage, latency.

### What you'll see in the LangSmith UI:
- **Run tree**: Each agent iteration as a nested span
- **Tool call nodes**: Input → Output for every tool invocation
- **Token usage**: Prompt tokens + completion tokens per step
- **Latency breakdown**: Time spent in LLM vs. tool execution
- **Errors**: Any tool or parsing failures highlighted in red

> 💡 Open [https://smith.langchain.com](https://smith.langchain.com) → Project `personal-ai-day5` to view traces.

---

## 🏁 Day 5 Summary & Key Takeaways

1.  **ReAct Pattern**: Agents loop through **Thought → Action → Observation** until they reach a final answer.
2.  **Tools as Interfaces**: Tools are your agent's "hands." Clear docstrings and Pydantic schemas are the most important part of tool design.
3.  **Modern vs. Classic**: Use `create_tool_calling_agent` for modern models (GPT-4, GPT-3.5) as it is more robust than text-based `create_react_agent`.
4.  **Agent Safety**: Always use `max_iterations` and `handle_parsing_errors=True` to prevent runaway agents or crashes.
5.  **Chain of Thought**: Agents can solve complex multi-step problems by chaining different tools together (e.g., search → calculate → summarize).
6.  **Observability**: LangSmith is essential for debugging agents — it reveals the "inner monologue" and tool-calling logic.

---

## 🌅 Next: Day 6 — Introduction to LangGraph

Tomorrow, we move beyond simple `AgentExecutor` to **LangGraph**. We'll learn how to build more complex, stateful, and custom-designed agent workflows using graphs, nodes, and edges.

**Day 6 Goals:**
- Define stateful workflows with Nodes and Edges
- Manage agent state across steps
- Implement custom control flow and cycles
- Build your first basic LangGraph agent!
